<a href="https://colab.research.google.com/github/BongWonHee/study_data_analystics/blob/main/codes/NLP/healthapp_review_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# refer : https://heytech.tistory.com/401

In [2]:
import pandas as pd

In [2]:
!pip install python-mecab-ko pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 26.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.

In [66]:
import pandas as pd
df_raw=pd.read_excel('./dataset_raw.xlsx')
df_raw[:2]


,app,review,rating
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1
1,다리 근육 운동 – 4주 프로그램,굿,5


## 전처리

In [67]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540076 entries, 0 to 540075
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   app     540076 non-null  object
 1   review  540074 non-null  object
 2   rating  540076 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.4+ MB


### 결측치 처리

In [68]:
df_raw_dropna=df_raw.dropna()
df_raw_dropna.isnull().sum()

app       0
review    0
rating    0
dtype: int64

### 자연어 전처리

#### 특정(상업용) 앱 리스트에서 제거

In [69]:
remove_app_list=pd.read_excel('./remove_app_list.xlsx')
remove_app_list[:5]

,app
0,캐시슬라이드 스텝업 - 걸음에 포인트를 더하다
1,만보기 - 걸음 계산기
2,딱 1주일 다이어트 습관 : 요요없는 건강한 다이어트
3,타임캐시 – 돈버는 어플
4,돈버는어플 - 캐시런


In [70]:
remove_app_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   app     106 non-null    object
dtypes: object(1)
memory usage: 976.0+ bytes


In [71]:
for remove_app in remove_app_list['app']:
    df_removed_app = df_raw_dropna[df_raw_dropna['app'] != remove_app]


In [72]:
df_removed_app=df_removed_app[500:90000]

### 형태소 적용과 불용어 처리

In [73]:
# 불용어 사전 불러오기
df_stopwords=pd.read_excel('./stopword_list.xlsx')
df_stopwords[:5]

,stopword
0,가까스로
1,가량
2,가령
3,가민
4,가민커넥트


In [74]:
from mecab import MeCab
mecab = MeCab()
stopwords = df_stopwords['stopword'].values
# 토크나이징 함수 정의 (글자가 한글자 이하이면 삭제, 불용어 삭제)
def tokenizer(raw, stopword=stopwords):
    result_list = list()
    for word in mecab.nouns(raw):
        if len(word) > 1 and word not in stopword :
            result_list.append(word)
    # 일부 케이스 (토큰 갯수 재한)
    # if len(result_list) <= 3 or len(result_list) < 12 :
    #   result_list
    return ' '.join(result_list) # return str

In [75]:
# mecab.nouns('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.')

In [76]:
# 형태 분석기 적용된 함수 실행
tokenizer('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.')

'운동'

In [77]:
df_removed_app['tokenizer_review']= df_removed_app['review'].apply(tokenizer)
df_removed_app

,app,review,rating,tokenizer_review
500,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,좋아요ㅎ,5,
501,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,nice app,5,
502,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,다 영어여서 쓰질 못해먹겠어!!!,1,영어
503,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,한글업데이트하면완벽,4,한글 업데이트 완벽
504,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,따라하기는 좋은데 매뉴만 한글이고 자세한 설명은 영어로 되서 보기 힘드네여,5,한글 설명 영어
...,...,...,...,...
89995,런데이 - 30분 달리기 PT,은동도 다이어트에도움이되는좋은앱입니다,5,은동 다이어트 도움
89996,런데이 - 30분 달리기 PT,좋으네용~~,5,
89997,런데이 - 30분 달리기 PT,좋아욘,5,
89998,런데이 - 30분 달리기 PT,너무좋아요,5,


### 유사어 대치

In [78]:
df_replace_list=pd.read_excel('./replace_list.xlsx')
df_replace_list[:3]

,before_replacement,after_replacement
0,S헬스,삼성헬스
1,LG폰,스마트폰
2,LG V10,스마트폰


In [79]:
def replace_word(tokenizer_review):
    replaced_review = str()
    for i in range(len(df_replace_list['before_replacement'])):
        try:
            # 치환할 단어가 있는 경우에만 데이터 치환 수행
            if df_replace_list['before_replacement'][i] in tokenizer_review:
                replaced_review = tokenizer_review.replace(df_replace_list['before_replacement'][i], df_replace_list['after_replacement'][i])
            else :
                replaced_review = tokenizer_review
        except Exception as e:
            print(f"Error 발생 / 에러명: {e}")

    return replaced_review

In [80]:
replace_word('LG V10 한글 업데이트 완벽 S헬스')

'LG V10 한글 업데이트 완벽 S헬스'

In [81]:
df_removed_app['replaced_review']=df_removed_app['tokenizer_review'].apply(replace_word)
df_removed_app[:10]

,app,review,rating,tokenizer_review,replaced_review
500,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,좋아요ㅎ,5,,
501,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,nice app,5,,
502,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,다 영어여서 쓰질 못해먹겠어!!!,1,영어,영어
503,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,한글업데이트하면완벽,4,한글 업데이트 완벽,한글 업데이트 완벽
504,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,따라하기는 좋은데 매뉴만 한글이고 자세한 설명은 영어로 되서 보기 힘드네여,5,한글 설명 영어,한글 설명 영어
505,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,회원탈퇴도 안되고 설명도 재대로 안되있는거같고 다 유료. 잖아요,1,회원 탈퇴 설명 유료,회원 탈퇴 설명 유료
506,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,강추,5,강추,강추
507,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,좋아요~,5,,
508,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,goog trainer app,5,,
509,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,Best app ever,5,,


In [ ]:
df_removed_app.to_csv('./preprocess_reviews.csv')

### 긍정과 부정 댓글 분리

In [20]:
condition_pasitive = df_removed_app['rating'] >= 3  # 긍정
condition_nagative = df_removed_app['rating'] < 3  # 부정

In [25]:
df_reviews_pasitive = df_removed_app[condition_pasitive]
df_reviews_nagative = df_removed_app[condition_nagative]


In [27]:
df_reviews_pasitive.isnull().sum()

app                 0
review              0
rating              0
tokenizer_review    0
replaced_review     0
dtype: int64

In [23]:
df_reviews_pasivive[:3]

,app,review,rating,tokenizer_review,replaced_review
500,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,좋아요ㅎ,5,,
501,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,nice app,5,,
503,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,한글업데이트하면완벽,4,한글 업데이트 완벽,


In [28]:
df_reviews_nagative.isnull().sum()

app                 0
review              0
rating              0
tokenizer_review    0
replaced_review     0
dtype: int64

In [29]:
df_reviews_nagative[:3]

,app,review,rating,tokenizer_review,replaced_review
502,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,다 영어여서 쓰질 못해먹겠어!!!,1,영어,
505,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,회원탈퇴도 안되고 설명도 재대로 안되있는거같고 다 유료. 잖아요,1,회원 탈퇴 설명 유료,
543,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,Not install download. . . .,2,,


In [30]:
df_reviews_pasivive.to_csv('./df_reviews_pasivive.csv')
df_reviews_nagative.to_csv('./df_reviews_nagative.csv')

### 긍정, 부정으로 전처리한 것을 csv로 저장

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## LDA 분석

In [ ]:
import numpy as np